In [1]:
import pandas as pd
import numpy as np
import hazm
import sklearn

In [40]:
data = pd.read_excel('label2.xlsx')

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40948 entries, 0 to 40947
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tweet       40935 non-null  object 
 1   label       1618 non-null   float64
 2   Unnamed: 2  11 non-null     object 
 3   Unnamed: 3  2 non-null      object 
dtypes: float64(1), object(3)
memory usage: 1.2+ MB


In [42]:
data = data.drop(['Unnamed: 2','Unnamed: 3'],axis=1)

In [53]:
data = data.dropna(axis=0)

In [15]:
with open('stopwords.txt',encoding="utf-8") as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Skygate
[nltk_data]     OPR5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

1495

In [19]:
import hazm
stemmer = hazm.Stemmer()
from hazm import word_tokenize

In [59]:
dataset = pd.DataFrame(columns=('Tweet', 'Label'))
for index, row in data[:1618].iterrows():
    title_body = row['tweet']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'Tweet': ' '.join(title_body_tokenized_filtered_stemmed),
        'Label': row['label']
    }

In [65]:
dataset

,Tweet,Label
0,میگ خوشتون نیاد عل کریم پستاشو دنبال میکردید ا...,1.0
1,ت مل بازیکنان حما ت مل قهر بشه ت مل قهر میشه ک...,0.0
2,پول کیرو جیب زده فشار افکار رسانه روشه برد صعو...,0.0
3,عادل فردوس پور کیرو مال بوته,0.0
4,گراز سرخ هماهنگ اعتراض بهانه واسه حذف ت مل کیر...,1.0
...,...,...
1613,کیرو فردوس پور ۹۰۰ نفر گایید جوون دارن خیابون ...,0.0
1614,#کیرو عل کریم بزدل فردوسی‌پور سناریو اهان نو #...,0.0
1615,دعو وحید امیر کیرو مشخص مصدوم وحید امیر ساختگ ...,0.0
1616,اوضاع عادل فردوسی‌پور مصاحبه کیرو پخ میکنه انگ...,0.0


In [66]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
vectorizer.fit(dataset['Tweet'])

TfidfVectorizer()

In [67]:
X = vectorizer.transform(dataset['Tweet'])

In [71]:
y = dataset['Label'].astype('uint8')

In [86]:
svmc = sklearn.svm.SVC()
svmc.fit(X, y)

SVC()

# Predicting unlabeled tweets

In [89]:
data = pd.read_excel('label2.xlsx')

In [106]:
data = data[1618:]

In [132]:
X = data['tweet'].dropna(axis=0)

In [135]:
dataset = pd.DataFrame(columns=('Tweet','Label'))
index=0
for row in X:
    title_body = row
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'Tweet': ' '.join(title_body_tokenized_filtered_stemmed)
    }
    index+=1

In [137]:
X = vectorizer.transform(dataset['Tweet'])

In [139]:
y_pred = svmc.predict(X)

In [156]:
data['tweet'].dropna(axis=0).reset_index(drop=True)

0        علی دایی، میرشاد ماجدی، دراگان اسکوچیچ، مهدی ط...
1        یادتونه که #عادل_فردوسی_پور شده بود اپوزیسیون ...
2            تو کیروش بهت گفت بزدل جرئت نداشتی جوابشو بدی 
3        هیچ کدوم نیستی-فقط بیسوادی!!! حدیث امام علی و ...
4        طرفداری و فوتفان که ازادی بیان میگن، جدی علی ک...
                               ...                        
39313    من تا اونجایی که میتونستم از کیروش هم حمایت می...
39314    دور قبل فقط یه بغل پای ساده طارمی و لازم داشتی...
39315    مشکلش چیه واقعا؟ داره نتیجه میگیره و همه‌مون م...
39316        فعلا پروفسور تون حذف میشه و تیم کیروش اول شد 
39317    طرفدارای کیروش میگن گروه ایران آسونه که ایران ...
Name: tweet, Length: 39318, dtype: object

In [157]:
dataset['Label'] = y_pred
dataset['Tweet'] = data['tweet'].dropna(axis=0).reset_index(drop=True)

In [158]:
dataset

,Tweet,Label
0,علی دایی، میرشاد ماجدی، دراگان اسکوچیچ، مهدی ط...,0
1,یادتونه که #عادل_فردوسی_پور شده بود اپوزیسیون ...,0
2,تو کیروش بهت گفت بزدل جرئت نداشتی جوابشو بدی,0
3,هیچ کدوم نیستی-فقط بیسوادی!!! حدیث امام علی و ...,0
4,طرفداری و فوتفان که ازادی بیان میگن، جدی علی ک...,0
...,...,...
39313,من تا اونجایی که میتونستم از کیروش هم حمایت می...,0
39314,دور قبل فقط یه بغل پای ساده طارمی و لازم داشتی...,0
39315,مشکلش چیه واقعا؟ داره نتیجه میگیره و همه‌مون م...,0
39316,فعلا پروفسور تون حذف میشه و تیم کیروش اول شد,0


In [159]:
dataset.to_excel('Output.xlsx')